<a href="https://colab.research.google.com/github/nicolaiberk/nlpdl_project/blob/main/BaselineBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install torch
!pip install sklearn

# code is based on Hauke Licht's CAP Model (https://colab.research.google.com/drive/1n7yHr0-lq-hmsXe2sqxLUhq7_4ejLH9o?usp=sharing#scrollTo=CiaoE0V8XZc6)

In [5]:
# define model
from transformers import BertForSequenceClassification
checkpoint_model = 'distilbert-base-german-cased'
model = BertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.lay

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/germanyPPRs.csv', engine="python")
df = df.reset_index()
df = df.dropna()
# df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [7]:
texts = list(df["text"])
labels = list(df["label"])

# there are probably better ways to do this
ulabels = list(set(labels))
label_dict = {}

for i in range(len(ulabels)):
  label_dict[str(ulabels[i])] = i

labels = [label_dict[str(l)] for l in labels]

In [8]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2, random_state=1234, stratify=labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=1234, stratify=train_labels)

print(f"No. obs    -- test: {len(test_labels)}; train: {len(train_labels)}; validation: {len(val_labels)};")
print(f"No. labels -- test: {len(set(test_labels))}; train: {len(set(train_labels))}; validation: {len(set(val_labels))};")

No. obs    -- test: 9267; train: 29652; validation: 7413;
No. labels -- test: 6; train: 6; validation: 6;


In [9]:
# load the tokenizer 
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-german-cased")

# tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
import torch
class PPRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PPRDataset(train_encodings, train_labels)
val_dataset = PPRDataset(val_encodings, val_labels)
test_dataset = PPRDataset(test_encodings, test_labels)

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }

In [12]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=3,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=64,
    # number of warmup steps for learning rate scheduler
    warmup_steps=500,
    # strength of weight decay
    weight_decay=0.01,     
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=250,
)


In [13]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

In [14]:
trainer.train()

Step,Training Loss
250,1.320000
500,0.125900
750,0.059300
1000,0.059900
1250,0.044200
1500,0.056500
1750,0.032200
2000,0.023900
2250,0.027600
2500,0.020100


TrainOutput(global_step=5562, training_loss=0.08852667922367727, metrics={'train_runtime': 8390.5231, 'train_samples_per_second': 0.663, 'total_flos': 1.841955243680563e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 327030, 'init_mem_gpu_alloc_delta': 271062528, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 713972, 'train_mem_gpu_alloc_delta': 805881344, 'train_mem_cpu_peaked_delta': 97480610, 'train_mem_gpu_peaked_delta': 6540903424})

## Validation performance

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
eval_res = trainer.evaluate()

Trainer is attempting to log a value of "[0.99541635 0.99073666 0.98983581 0.99792674 0.99723672 0.99641577]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.99693956 0.98422436 0.99842271 0.99723757 0.997543   0.99681275]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.99389779 0.9973357  0.98139535 0.99861687 0.99693063 0.99601911]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1311 1126  645 1446 1629 1256]" of type <class 'numpy.ndarray'> for key "eval/n" as a scalar. This invocation of Tensorboard's writer.add_sc

In [16]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in label_dict.items()]
evaluated["class"] = [k for k,v in label_dict.items()]
evaluated["f1"] = eval_res["eval_f1"]
evaluated["precision"] = eval_res["eval_precision"]
evaluated["recall"] = eval_res["eval_recall"]
evaluated["n"] = eval_res["eval_n"]

In [17]:
evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
3,3,SPD,0.997927,0.997238,0.998617,1446
4,4,Linke,0.997237,0.997543,0.996931,1629
5,5,FDP,0.996416,0.996813,0.996019,1256
0,0,Greens,0.995416,0.996940,0.993898,1311
1,1,Union,0.990737,0.984224,0.997336,1126
2,2,AfD,0.989836,0.998423,0.981395,645


In [18]:
trainer.save_model(os.path.join("trained", "distilbert_cased", "results"))

## Test set performance

In [19]:
eval_res = trainer.predict(test_dataset)

In [20]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in label_dict.items()]
evaluated["class"] = [k for k,v in label_dict.items()]
evaluated["f1"] = eval_res.metrics["eval_f1"]
evaluated["precision"] = eval_res.metrics["eval_precision"]
evaluated["recall"] = eval_res.metrics["eval_recall"]
evaluated["n"] = eval_res.metrics["eval_n"]

evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
5,5,FDP,0.997454,0.996819,0.998089,1570
4,4,Linke,0.997301,0.997056,0.997545,2037
3,3,SPD,0.997236,0.996135,0.998340,1807
0,0,Greens,0.995409,0.998158,0.992674,1638
1,1,Union,0.991507,0.988011,0.995028,1408
2,2,AfD,0.990037,0.994994,0.985130,807


In [26]:
pd.crosstab(eval_res.label_ids, eval_res.predictions.argmax(-1))

col_0,0,1,2,3,4,5
row_0,,,,,,
0,1626,2,2,3,2,3
1,0,1401,2,2,2,1
2,1,8,795,0,2,1
3,2,1,0,1804,0,0
4,0,4,0,1,2032,0
5,0,2,0,1,0,1567
